### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 3 - Digital Video Processing
In this assignment, you will use OpenCV and FFmpeg to implement very basic video editing functions. These tasks include:

1. Create a slide show (as a video) from images, and optionally create the slideshow as greyscale video.
2. Extract the audio track from a video file.
3. Replace the audio track in a video file.
4. Combine two or more videos into one video file.
5. Blend an image (fade-in/fade-out) with a video.
6. Blend two videos into one video (video collage).

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.

## Setup

For general installation instructions, please refer to the ressources given for all the assignments [in Moodle](https://moodle.univie.ac.at/course/view.php?id=164140#section-12).

If the cell below executes without error, you can start the assignment!

In [259]:
# -------- Imports --------
# Please do not change the contents of this cell!

# Imports required by us.
import cv2              # opencv-python
import ffmpeg           # ffmpeg-python
import subprocess   # for calling local executables such as ffmpeg.exe
import pandas as pd  # pandas
from fractions import Fraction as frac # simplifying fractions

In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [260]:
# Place your own imports here.
import os
import sys

import IPython
from IPython.display import HTML

In [261]:
# Place any helper functions, global variables and classes here.

# For example:the function you need to play back a mp4-video file.
# You may use this function to display the videos in your definition file during the assessment for demoing the solutions.
def VideoPlayer(videoFile: str) -> None:
    
    IPython.display.display(
        HTML("""
            <video alt="test" controls>
            <source src={} type="video/mp4">
            </video>
        """.format(videoFile))
    )
    
    return

In [262]:
#VideoPlayer("./resources/video/sample1.mp4")


## Task 3.1: Create a slide show (Video) from multiple images and convert it to greyscale 

In [263]:
# https://learnopencv.com/read-write-and-display-a-video-using-opencv-cpp-python/
# https://docs.opencv.org/4.x/dd/d9e/classcv_1_1VideoWriter.html

def CreateVideoFromImages(inImgLibFolder, imageFormat, durationInSec, convertToGreyScale, outFolder, outVideo):
    if not os.path.exists(outFolder):
        os.makedirs(outFolder)
    outputFilepath = os.path.join(outFolder, outVideo)
    
    # OpenCV VideoWriter with MP4 and 1280x720
    # Duration established with framerate
    videoWriter = cv2.VideoWriter(outputFilepath, cv2.VideoWriter_fourcc('m','p','4','v'), 1/durationInSec, (1280, 720))
    
    if os.path.isdir(inImgLibFolder):
        for filename in os.listdir(inImgLibFolder):
            filepath = os.path.join(inImgLibFolder, filename)
            name, extension = os.path.splitext(filename)
            if os.path.isfile(filepath) and extension.upper() in [('.' + imageFormat.upper())]: #just take these extensions:
                
                # Read Image
                img = cv2.imread(filepath)
                
                # Resize to fit
                img = cv2.resize(img, (1280, 720)) 
                
                # Greyscale
                if convertToGreyScale:
                    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # grey
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) # and back to RGB
                
                # Append to videoWriter
                videoWriter.write(img)
    
    videoWriter.release()


In [264]:
# CreateVideoFromImages("./task1/images/", "JPG", 2, True, "./task1/output/", "outTask1Video.mp4")
# VideoPlayer("./task1/output/outTask1Video.mp4")

## Task 3.2: Extract the Audio Track from a Video File 

In [265]:
# from my MRS Assignment 2
# https://superuser.com/questions/268985/remove-audio-from-video-file-with-ffmpeg

def SplitAudioVideoTracks(inVideo, outFolder, outVideoTrack, outAudioTrack) -> None:
    if not os.path.exists(outFolder):
        os.makedirs(outFolder)
    outAudioFilepath = os.path.join(outFolder, outAudioTrack)
    outVideoFilepath = os.path.join(outFolder, outVideoTrack)
    
    # extract wav with ffmpeg
    subprocess.run(['ffmpeg', '-i', inVideo, '-acodec' ,'pcm_s16le', '-y', outAudioFilepath])#, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT) # hidden output
    
    # copy video w_o_ audio with ffmpeg
    # -an for no audio
    subprocess.run(['ffmpeg', '-i', inVideo, '-c' ,'copy', '-an', '-y', outVideoFilepath])#, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

In [266]:
#SplitAudioVideoTracks("./resources/video/sample1.mp4", "./task2/output/", "outTask2VideoTrack.mp4", "outTask2AudioTrack.wav")

## Task 3.3: Replace the Audio Track in a Video File

In [267]:
# https://superuser.com/questions/1137612/ffmpeg-replace-audio-in-video

def AddOrReplaceAudio(inVideo, inAudio, outFolder, outVideo) -> None:
    if not os.path.exists(outFolder):
        os.makedirs(outFolder)
    outVideoFilepath = os.path.join(outFolder, outVideo)
    
    # copy video
    # map input audio as audiostream
# "-shortest" parameter
    subprocess.run(['ffmpeg', '-i', inVideo, '-i', inAudio, '-c:v' ,'copy', '-map', '0:v:0', '-map', '1:a:0', '-shortest', '-y', outVideoFilepath])#, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    

In [268]:
# AddOrReplaceAudio("./resources/video/sample1.mp4", "./resources/audio/Amazon.mp3", "./task3/output/", "outTask3Video.mp4")
# VideoPlayer("./task3/output/outTask3Video.mp4")

## Task 3.4: Combine Videos

In [269]:
# https://docs.opencv.org/4.x/d4/d15/group__videoio__flags__base.html
# https://stackoverflow.com/questions/61659346/how-to-get-4-character-codec-code-for-videocapture-object-in-opencv
# https://ffmpeg.org/ffprobe.html
# https://stackoverflow.com/questions/73142995/trying-to-fetch-all-audio-streams-with-ffmpeg-python

def VideoMetadataExtractor(inVideoLibFolder) -> pd.DataFrame:
    dataFrame = pd.DataFrame(columns=['filename', 'vCodec', 'vcodecID', 'vDur', 'vFPS', 'vHeight', 'vWidth', 'aCodec', 'acodecID', 'aChannels', 'aSamplerate', 'aBitrate'])
    
    if os.path.isdir(inVideoLibFolder):
        for filename in sorted(os.listdir(inVideoLibFolder)):
            filepath = os.path.join(inVideoLibFolder, filename)
            name, extension = os.path.splitext(filename)
            if os.path.isfile(filepath):# and extension.upper() in ['.MP4']:
                
                # Video with OpenCV
                videoCapture = cv2.VideoCapture(filepath)
                vcodecID = int(videoCapture.get(cv2.CAP_PROP_FOURCC))
                vCodec = vcodecID.to_bytes(4, byteorder=sys.byteorder).decode() # codecID is hex for the four-character-code
                vFPS = videoCapture.get(cv2.CAP_PROP_FPS)
                vDur = videoCapture.get(cv2.CAP_PROP_FRAME_COUNT) / vFPS
                vHeight = videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)
                vWidth = videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)
                
                # Audio with FFProbe
                probe = ffmpeg.probe(filepath)
                audioStream = None
                for stream in probe['streams']:
                    if stream['codec_type'] == 'audio':
                        audioStream = stream
                        break
                
                newRow = pd.DataFrame.from_records([{
                    'filename': filename,
                    'vCodec': vCodec,
                    'vcodecID': vcodecID,
                    'vDur': vDur,
                    'vFPS': vFPS,
                    'vHeight': vHeight,
                    'vWidth': vWidth,
                    'aCodec': audioStream['codec_name'],
                    'acodecID': audioStream['codec_tag_string'],
                    'aChannels': int(audioStream['channels']),
                    'aSamplerate': int(audioStream['sample_rate']),
                    'aBitrate': int(audioStream['bit_rate'])
                }])
                dataFrame = pd.concat([dataFrame, newRow])
                
    dataFrame.reset_index(drop=True, inplace=True)
    return dataFrame

In [270]:
# display(VideoMetadataExtractor("./resources/video/"))

In [271]:
#https://creatomate.com/blog/how-to-join-multiple-videos-into-one-using-ffmpeg
#https://ffmpeg.org/ffmpeg-filters.html#concat
#https://superuser.com/questions/1806388/ffmpeg-error-input-link-parameters-do-not-match-the-corresponding-output-link-pa

def CombineVideos(inVideoLibFolder, outFolder, outVideo) -> None:
    if not os.path.exists(outFolder):
        os.makedirs(outFolder)
    outVideoFilepath = os.path.join(outFolder, outVideo)
    
    # initialize ffmpeg command
    cmd = ['ffmpeg']
    numberVideos = 0
    
    if os.path.isdir(inVideoLibFolder):
        for filename in sorted(os.listdir(inVideoLibFolder)):
            filepath = os.path.join(inVideoLibFolder, filename)
            name, extension = os.path.splitext(filename)
            if os.path.isfile(filepath):# and extension.upper() in ['.MP4']:
                numberVideos += 1
                cmd.append('-i')
                cmd.append(filepath)
    
    
    # here, the complex filter is manually created
    cmd.append('-filter_complex')
    filter = ''
    for i in range(0, numberVideos):
        x = '['+str(i)+':v]scale=1280:720[v'+str(i)+'];' #scaling all videos to same size (and given a new name)
        filter += x
    for i in range(0, numberVideos):
        x = '[v' + str(i) + '][' + str(i) + ':a]' #appending all streams to complex filter
        filter += x
    filter += 'concat=n=' + str(numberVideos) + ':v=1:a=1' #adding the concat, n, and outputstreams
    cmd.append(filter)
    
    cmd.append('-vsync')
    cmd.append('vfr')
    cmd.append('-y')
    cmd.append(outVideoFilepath)
     
    print(cmd)
    subprocess.run(cmd)#, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)


In [272]:
# CombineVideos("./resources/video/", "./task4/output/", "outTask4.mp4")

## Task 3.5: Blend an Image in a Video

In [273]:
# when i remove the fade, the picture shows
# the picture just wont show with the fade.....

#https://ffmpeg.org/ffmpeg-filters.html

def AddFadingImage(inVideo, inImg, time, outFolder, outVideo) -> None:
    p = 1 #fade-duration
    totalDuration = 2 * p + time
    
    if not os.path.exists(outFolder):
        os.makedirs(outFolder)
    outVideoFilepath = os.path.join(outFolder, outVideo)
    
    cmd = ['ffmpeg']
    cmd.append('-i')
    cmd.append(inVideo)
    cmd.append('-i')
    cmd.append(inImg)
    
    cmd.append('-filter_complex')
    
    filter  = f'[0:v]scale=1280:720[vid];' #scale video
    filter += f'[1:v]scale=1280:720,' #scale
    filter += f'fade=t=in:st=0:d={p}:alpha=1,' #fade in
    filter += f'fade=t=out:st={time + p}:d={p}:alpha=1[img];' #fade out image
    filter += f"[vid][img]overlay=0:0:enable='between(t,0,{totalDuration})'" #only enable overlay in this time
    cmd.append(filter)
    
    cmd.append('-y')
    cmd.append(outVideoFilepath)
     
    print(cmd)
    subprocess.run(cmd)#, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

In [274]:
# AddFadingImage("./resources/video/sample2.mp4", "./resources/images/elephant14m.JPG", 2, "./task5/output/", "outTask5.mp4")

## Task 3.6: Create a Video Collage - blend two videos into one

In [275]:
#https://www.baeldung.com/linux/ffmpeg-stitch-videos-horizontally

def VideoClipMixer(inVideo1, inVideo2, layout, outFolder, outVideo) -> None: 
    if not os.path.exists(outFolder):
        os.makedirs(outFolder)
    outVideoFilepath = os.path.join(outFolder, outVideo)
    
    cmd = ['ffmpeg']
    cmd.append('-i')
    cmd.append(inVideo1)
    cmd.append('-i')
    cmd.append(inVideo2)
    
    cmd.append('-filter_complex')
    
    if layout == 'column':
        filter  = f'[0:v]scale=1280:720/2[v1];'
        filter += f'[1:v]scale=1280:720/2[v2];'
        filter += f"[v1][v2]vstack"
        cmd.append(filter)
        
    if layout == 'row':
        filter  = f'[0:v]scale=1280/2:720[v1];'
        filter += f'[1:v]scale=1280/2:720[v2];'
        filter += f"[v1][v2]hstack"
        cmd.append(filter)
        
    cmd.append('-vsync')
    cmd.append('vfr')
    cmd.append('-shortest')
    cmd.append('-y')
    cmd.append(outVideoFilepath)
     
    print(cmd)
    subprocess.run(cmd)#, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)


In [276]:
#VideoClipMixer("./resources/video/sample2.mp4", "./resources/video/sample3.avi", 'row', "./task6/output/", "outTask6.mp4")